<a href="https://colab.research.google.com/github/mesha4545a/Regressed-Classification-Project-/blob/main/ML_Weekend_Project_Room11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing needed libraries**


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import *
sns.set_palette('Set2')

#**Importing Data**

In [ ]:
# Starter Code --DO NOT ADJUST--
col_names = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "ooccupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country", "target"]
training = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", names=col_names).drop("race", axis=1)
test = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", names=col_names, skiprows=1).drop("race", axis=1)

training["target"] = training["target"].apply(lambda x: x.strip())
test["target"] = test["target"].apply(lambda x: x.strip()[:-1])




#**EDA**

###**Data Understanding**

####Gathering information about the dataset

In [ ]:
training.info()

####Dataset Shape

In [ ]:
training.shape

####Describing Categorical Data

In [ ]:
 training.describe(exclude='number')

####Viewing some basic statistical details‏

In [ ]:
 training.describe()

####Checking Null Values

In [ ]:
training.isnull().sum()

####Value Count of All Categorical Data

#####workclass

In [ ]:
training.workclass.value_counts()

#####education

In [ ]:
training.education.value_counts()

#####marital_status

In [ ]:
training.marital_status.value_counts()

#####ooccupation

In [ ]:
training.ooccupation.value_counts()

#####relationship

In [ ]:
training.relationship.value_counts()

#####sex

In [ ]:
training.sex.value_counts()

#####native_country

In [ ]:
training.native_country.value_counts()

#####target

In [ ]:
training.target.value_counts()

###**Data Visualization**

Training Dataset Pair Plot Chart

In [ ]:
sns.pairplot(training.sample(1000), hue='target')
plt.title('Traning DataSet')
plt.show()

- Plotting pairwise relationships in the training dataset
- With excluding the ‘target’ (income) as a hue

Differentiations of Income Based on Gender

In [ ]:
fig=plt.figure(figsize=(5,5))
sns.countplot(data=training, x='target', hue='sex')
plt.title('Income per Sex')
plt.show()

As it’s shown in the chart,
- The majority of the workers are males, and they are representing the majority of both income’s categories.
- The majority of the workers (and the most of the females), are getting an income for <=50 a year.

In [ ]:
fig=plt.figure(figsize=(20,5))
sns.countplot(data=training,x='education',hue='target')
plt.legend(loc='upper right')
plt.show()

Relationship between Income and Working-hours

In [ ]:
fig=plt.figure(figsize=(20,5))
sns.lineplot(x="age", y="hours_per_week",hue="target", data=training)

plt.title('Income rate for Income and Working-hours')
plt.legend(loc='upper right')
plt.show()

**There are many insights can be found in this chart such as:**
- More working-hours = more income 
- Workers with high income work more than 40hrs a week, and they start to work fewer hours at the age of 60. Also, less-income-workers work less than 40hrs a week.
- Something strange about the workers, regarding how they are working for longer hours in their 80’s than they start-working ages, and even longer than the hours of the less-income-workers as well. 

Relationship between Income and Workers’ Occupation

In [ ]:
fig=plt.figure(figsize=(20,5))
sns.countplot(y='ooccupation',hue='target',data=training)
plt.title("Target by occupation")
plt.show()

**The clearest insights that can be seen from this chart are:**

- Eexec-managerial and Prof-specialty workers are usually having a high income.
- Adm-clerical, craft-repair, other-service workers are usually having less income.
- Haigh armed forces and priv-house-serv workers are usually having the lowest income among all the other workers.

High Income by Occupation

First we get the dummy of target column

In [ ]:
target_d=pd.get_dummies(training.target)


In [ ]:
target_d

Then we add it to a new dataframe so we can get the pivot table

In [ ]:
df=pd.DataFrame(training)
df['<=50K']=target_d['<=50K']
df['>50K']=target_d['>50K']

In [ ]:
col=['<=50K','>50K']
training.drop(columns=col,axis=1,inplace=True)

We create the pivot table

In [ ]:
dfpO=df.pivot_table(index='ooccupation',aggfunc='mean',values=['<=50K','>50K'])
dfpO

In [ ]:
sorted_m50kO=dfpO.sort_values(['>50K'],ascending=False).index
sorted_m50kO

In [ ]:
sorted_l50kO=dfpO.sort_values(['<=50K'],ascending=False).index
sorted_l50kO

In [ ]:
fig=plt.figure(figsize=(20,5))
sns.barplot(y=dfpO.index,x=dfpO['>50K'], order=sorted_m50kO)
plt.title('High Income rate occopation')
plt.show()

- Eexec-managerial and Prof-specialty workers are usually having a high income 

Low Income rate per occpoation

In [ ]:
fig=plt.figure(figsize=(20,5))
sns.barplot(y=dfpO.index,x=dfpO['<=50K'], order=sorted_l50kO)
plt.title('low Income rate per occopation')
plt.show()

- Adm-clerical, craft-repair, other-service workers are usually having less income.
- Haigh armed forces and priv-house-serv workers are usually having the lowest income among all the other workers

How Income is effected by the education level

We create a pivot table

In [ ]:
dfp=df.pivot_table(index='education',aggfunc='mean',values=['<=50K','>50K'])

In [ ]:
dfp

We should sort values for good visualization

In [ ]:
sorted_m50k=dfp.sort_values(['>50K'],ascending=False).index
sorted_m50k

Here we visualize the High income rate per education level.

In [ ]:
fig=plt.figure(figsize=(20,5))
sns.barplot(y=dfp.index,x=dfp['>50K'], order=sorted_m50k)
plt.title('High Income rate per education level')
plt.show()

In the above chart you can see what education level have the highest probability to have high income (more than 50K $)

We do the same process to get result of low income rate

In [ ]:
sorted_l50k=dfp.sort_values(['<=50K'],ascending=False).index
sorted_l50k

In [ ]:
fig=plt.figure(figsize=(20,5))
sns.barplot(y=dfp.index,x=dfp['<=50K'], order=sorted_l50k)
plt.title('Low Income rate per education level')
plt.show()

It is obvious in the above chart that as people have low level of education they are most likely to have low income rate (less than 50K $)

Income relatiionship with Capital Gain and Capital Loss

In [ ]:
fig=plt.figure(figsize=(20,5))

sns.scatterplot(data=training, x='capital_gain',y='capital_loss',hue='target')

plt.title('Income rate per capital gain/loss')
plt.legend(loc='upper right')
plt.show()

There are some interesting points regarding this chart:
- Some of people who making only capital loss have high income!
- There are people who have a huge capital gain with more than 30,000 $ and still considered as they have few income in the dataset. That is not consist and not logic.
- It is noticable that there is an outlier in the capital gain so we will check for all outliers

Is there any outlier?

In [ ]:
fig=plt.figure(figsize=(10,7))
sns.boxplot(data=training)
plt.show()

As seen, "fnlwgt" and "capital_gain" features have outliers and we should remove them in the next stage (Data Preperation)

#**Data Preperation**

###**Data Cleansing**

####**Removing outliers**

In [ ]:
training=training[(training['fnlwgt']<training['fnlwgt'].quantile(.95))&(training['capital_gain']<training['capital_gain'].quantile(.95))]

In [ ]:
fig=plt.figure(figsize=(15,7))
sns.boxplot(data=training)
plt.show()

In [ ]:
training.isnull().sum()

####**Removing inconsistent‏ features**


We have inconsistent‏ data from three features. These are :
- "Workclass"
- "ooccupation"
- "Native_country"


In [ ]:
training=training[training['workclass']!=' ?']
training=training[training['ooccupation']!=' ?']
training=training[(training['native_country']!=' ?') & (training['native_country']!= ' Holand-Netherlands' )]

now we check the data are they removed or not?

In [ ]:
training.native_country.value_counts()

####**Replacing the Target values to (1,0)**

In [ ]:
training["target"] = training["target"].replace(['<=50K','>50K'],(1,0))
test["target"] = test["target"].replace(['<=50K','>50K'],(1,0))

####**Cleaning the Test dataset to fit it into the model**

#####**Explorating Data**

In [ ]:
test.isnull().sum()

In [ ]:
test.workclass.value_counts()

In [ ]:
test.ooccupation.value_counts()

In [ ]:
test.native_country.value_counts()

#####**Removing outliers**

In [ ]:
test=test[(test['fnlwgt']<test['fnlwgt'].quantile(.95))&(test['capital_gain']<test['capital_gain'].quantile(.95))]

In [ ]:
fig=plt.figure(figsize=(15,7))
sns.boxplot(data=test)
plt.show()

In [ ]:
training.isnull().sum()

#####**Removing inconsistent‏ features**


We have inconsistent‏ data from three features. These are :
- "Workclass"
- "ooccupation"
- "Native_country"


In [ ]:
test=test[test['workclass']!=' ?']
test=test[test['ooccupation']!=' ?']
test=test[(test['native_country']!=' ?') & (test['native_country']!= ' Holand-Netherlands' )]

now we check the data are they removed or not?

In [ ]:
test.native_country.value_counts()

###**Features Extraction**

We will add two features as we believe that will increase model scores

Adding a new feature - Age Category

In [ ]:
training.loc[training['age']<=35,'Age Category']='young'
training.loc[(training['age']>35) & (training['age']<=60),'Age Category']='mature'
training.loc[training['age']>60,'Age Category']='old'

In [ ]:
training.sample(5)

In [ ]:
fig=plt.figure(figsize=(10,5))
sns.countplot(data=training,x='Age Category',hue='target')
plt.show()

As seen in the previous chart, young people are more likely to have low income. That is sence as usually people at the begining of their lives focus more on studying rather than having a job.

In addition, it seems there is a correlation between our new feature and the target and that might increase the model score

Adding a new feature - Education Category

In [ ]:
training.loc[(training['education']==' 1st-4th')|
             (training['education']==' 5th-6th')|
             (training['education']==' 7th-8th')|
             (training['education']==' 9th')|
             (training['education']==' 10th')|
             (training['education']==' 11th')|
             (training['education']==' 12th')|
             (training['education']==' HS-grad')|
             (training['education']==' Preschool'),'Education Category']='general education'

training.loc[(training['education']==' Bachelors')|
             (training['education']==' Assoc-acdm')|
             (training['education']==' Assoc-voc')|
             (training['education']==' Some-college'),'Education Category']='undergraduate'



training.loc[(training['education']==' Masters')|
             (training['education']==' Prof-school')|
             (training['education']==' Doctorate'),'Education Category']='postgraduate'

In [ ]:
training.sample(3)

In [ ]:
fig=plt.figure(figsize=(10,5))
sns.countplot(data=training,x='Education Category',hue='target')
plt.show()

In the above chart it is shown how the new feature is correlated with the target.

Usually postgraduate people have high level jobs and their income is much higher than what undergraduate and general education people have.

####**Test Features Extraction**

We will add two features in Test dataset so there will be no confilct in the model

Adding a new feature - Age Category

In [ ]:
test.loc[test['age']<=35,'Age Category']='young'
test.loc[(test['age']>35) & (test['age']<=60),'Age Category']='mature'
test.loc[test['age']>60,'Age Category']='old'

In [ ]:
test.sample(5)

Adding a new feature - Education Category

In [ ]:
test.loc[(test['education']==' 1st-4th')|
             (test['education']==' 5th-6th')|
             (test['education']==' 7th-8th')|
             (test['education']==' 9th')|
             (test['education']==' 10th')|
             (test['education']==' 11th')|
             (test['education']==' 12th')|
             (test['education']==' HS-grad')|
             (test['education']==' Preschool'),'Education Category']='general education'

test.loc[(test['education']==' Bachelors')|
             (test['education']==' Assoc-acdm')|
             (test['education']==' Assoc-voc')|
             (test['education']==' Some-college'),'Education Category']='undergraduate'



test.loc[(test['education']==' Masters')|
             (test['education']==' Prof-school')|
             (test['education']==' Doctorate'),'Education Category']='postgraduate'

In [ ]:
test.sample(5)

###**Categorical Encoding (Getting Dummies)**

Here we get the dummy of our data

In [ ]:
training=pd.get_dummies(data=training)
test=pd.get_dummies(data=test)

##**Dataset spliting**

This is the spliting from the given starter code

In [ ]:
# Create Train
X_train = training.drop("target", axis=1)
y_train = training["target"]

# Create Test
X_test = test.drop("target", axis=1)
y_test = test["target"]


Here we will check if the given spliting is suitable or not for our models.

Identefing features correlation

In [ ]:
corr=training.corr()


In [ ]:
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
extracted_features=pd.Series(corr[(corr['target']>=0.2)|(corr['target']<=-0.2)].index)

In [ ]:
extracted_features

In [ ]:
X_train=training.loc[:,(extracted_features)]
X_train.drop('target',axis=1,inplace=True)

X_test=test.loc[:,(extracted_features)]
X_test.drop('target',axis=1,inplace=True)

In [ ]:
X_train

#**Data Modeling**

###**Model Selection**

#### Here we will list all models and clarify what type of models is more sutiable for our dataset.

####**Baseline**

In [ ]:
def baseline_model(n_predictions, value_to_predict):
    baseline_preds = []
    for i in range(n_predictions):
        baseline_preds.append(value_to_predict)
    return pd.Series(baseline_preds)

n_predictions = len(y_test)
baseline_value = y_train.value_counts().index[0]

baseline_preds = baseline_model(n_predictions, baseline_value)
baseline_preds

In [ ]:
bl_acc = accuracy_score(y_test , baseline_preds)
print('BaseLine Accurecy Score = ' , bl_acc)

In [ ]:
cf = confusion_matrix(y_test, baseline_preds)
group_names = ['TP','FP','FN','TN']
group_counts = ['{0:0.0f}'.format(value) for value in cf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf.flatten()/np.sum(cf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ticks_labels = ['A', 'B']

plt.figure(figsize=(5,5))
sns.heatmap(cf, annot=labels, fmt='', cmap='Pastel2', yticklabels=ticks_labels , xticklabels= ticks_labels);
plt.title("Baseline")
plt.ylabel('Actual')
plt.xlabel('Predicted')
sns.set(font_scale=1);

####**StandardScaler Model**

In [ ]:
# standrd = StandardScaler()
# training = standrd.fit_transform(training)
# # training

# test = standrd.fit_transform(test)
# # test

####**Logistic Regression Model**

In [ ]:
lr = LogisticRegression(max_iter=10000)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
lr_preds1=lr.predict(X_test)

In [ ]:
lr_acc1 = accuracy_score(y_test, lr_preds1)
print('Logistic Regression Accurecy Score = ' , lr_acc1)

In [ ]:
cf = confusion_matrix(y_test, lr_preds1)
group_names = ['TP','FP','FN','TN']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf.flatten()/np.sum(cf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ticks_labels = ['A', 'B']

plt.figure(figsize=(5,5))
sns.heatmap(cf, annot=labels, fmt='', cmap='Pastel2', yticklabels=ticks_labels , xticklabels= ticks_labels);
plt.title("Logistic Regression")
plt.ylabel('Actual')
plt.xlabel('Predicted')
sns.set(font_scale=1);

####**SGD Classifier Model**

In [ ]:
sgdc = SGDClassifier(max_iter=10000)

In [ ]:
sgdc.fit(X_train, y_train)

In [ ]:
sgd_preds1=sgdc.predict(X_test)

In [ ]:
sgd_acc1 = accuracy_score(y_test, sgd_preds1)
print('SGD Classifier Accurecy Score = ' , sgd_acc1)

In [ ]:
cf = confusion_matrix(y_test, sgd_preds1)
group_names = ['TP','FP','FN','TN']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf.flatten()/np.sum(cf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ticks_labels = ['A', 'B']

plt.figure(figsize=(5,5))
sns.heatmap(cf, annot=labels, fmt='', cmap='Pastel2', yticklabels=ticks_labels , xticklabels= ticks_labels);
plt.title("SGD Classifier")
plt.ylabel('Actual')
plt.xlabel('Predicted')
sns.set(font_scale=1);

####**KNeighbors Classifier Model**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
knn_perds1 = knn.predict(X_test)

In [ ]:
knn_acc1 = accuracy_score(y_test, knn_perds1)
print('KNeighbors Classifier Accurecy Score = ' , knn_acc1)

In [ ]:
cf = confusion_matrix(y_test, knn_perds1)
group_names = ['TP','FP','FN','TN']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf.flatten()/np.sum(cf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ticks_labels = ['A', 'B']

plt.figure(figsize=(5,5))
sns.heatmap(cf, annot=labels, fmt='', cmap='Pastel2', yticklabels=ticks_labels , xticklabels= ticks_labels);
plt.title("KNeighbors Classifier")
plt.ylabel('Actual')
plt.xlabel('Predicted')
sns.set(font_scale=1);

####**Models Acuurecy Cmparisions**

In [ ]:
pd.DataFrame([bl_acc, lr_acc1 , sgd_acc1, knn_acc1]).plot.bar()
plt.xticks(np.arange(4),('Baseline','LogicReg','SGD','KNN'))
plt.legend().remove()
plt.ylim(0.6, 0.9)
plt.annotate = True
plt.ylabel('Accuracy')
plt.xlabel('Model')
plt.xticks(rotation = 0)
plt.title('Comparing Model Accuracy');

**As we can see from the chart,**
- The accurecy score of Baseline Model is

In [ ]:
bl_acc

- And all of our used models socres beated the Baseline score!

In [ ]:
print('Logistic Regression = ' , lr_acc1 ,  '\nSGD = ',sgd_acc1, '\nKNeighbors=', knn_acc1 )

- For our 1st predection, we noticed that the Logistic Regression accurecy score is the highest one.

    So, we assumed that it is the most accurate model to go with but we will test that assumption by doing the optimaization part.

#**Model Optimization**

###**GridSearchCV**

####**Logistic Regression Model**

In [ ]:

p_grid = {'fit_intercept': [True, False],
          'penalty':['l1', 'l2', 'elasticnet'],
          'multi_class':['auto', 'ovr', 'multinomial'],
          'warm_start':[True,False]
          

}

grid = GridSearchCV(lr, p_grid, cv=10,n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
lr = LogisticRegression(max_iter=10000,
                        fit_intercept= False,
                        multi_class='auto',
                        penalty='l2',
                        warm_start= True)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
lr_preds2=lr.predict(X_test)

In [ ]:
lr_acc2 = accuracy_score(y_test, lr_preds2)
print('Logistic Regression Accurecy Score After GridSearchCV Optimization = ' , lr_acc2)

In [ ]:
cf = confusion_matrix(y_test, lr_preds2)
group_names = ['TP','FP','FN','TN']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf.flatten()/np.sum(cf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ticks_labels = ['A', 'B']

plt.figure(figsize=(5,5))
sns.heatmap(cf, annot=labels, fmt='', cmap='Pastel2', yticklabels=ticks_labels , xticklabels= ticks_labels);
plt.title("Logistic Regression After GridSearchCV Optimization")
plt.ylabel('Actual')
plt.xlabel('Predicted')
sns.set(font_scale=1);

####**SGD Classifier Model**

In [ ]:
p_grid = {'fit_intercept': [True, False],
          'penalty':['l1', 'l2', 'elasticnet'],
          

}

grid = GridSearchCV(sgdc, p_grid, cv=10,n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
sgdc = SGDClassifier(max_iter=10000,
                     fit_intercept= False,
                     penalty='elasticnet')

In [ ]:
sgdc.fit(X_train, y_train)

In [ ]:
sgd_preds2=sgdc.predict(X_test)

In [ ]:
sgd_acc2 = accuracy_score(y_test, sgd_preds2)
print('SGD Classifier Accurecy Score After GridSearchCV Optimization = ' , sgd_acc2)

In [ ]:
cf = confusion_matrix(y_test, sgd_preds2)
group_names = ['TP','FP','FN','TN']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf.flatten()/np.sum(cf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ticks_labels = ['A', 'B']

plt.figure(figsize=(5,5))
sns.heatmap(cf, annot=labels, fmt='', cmap='Pastel2', yticklabels=ticks_labels , xticklabels= ticks_labels);
plt.title("SGD Classifier After GridSearchCV Optimization")
plt.ylabel('Actual')
plt.xlabel('Predicted')
sns.set(font_scale=1);

####**KNeighbors Classifier Model**

In [ ]:
p_grid = {'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}

grid = GridSearchCV(knn, p_grid, cv=10,n_jobs=-1)




In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
KNeighborsClassifier(algorithm= 'brute')
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
knn_perds2 = knn.predict(X_test)

In [ ]:
knn_acc2 = accuracy_score(y_test, knn_perds2)
print('KNeighbors Classifier Accurecy Score After GridSearchCV Optimization = ' , knn_acc2)

In [ ]:
cf = confusion_matrix(y_test, knn_perds2)
group_names = ['TP','FP','FN','TN']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf.flatten()/np.sum(cf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ticks_labels = ['A', 'B']

plt.figure(figsize=(5,5))
sns.heatmap(cf, annot=labels, fmt='', cmap='Pastel2', yticklabels=ticks_labels , xticklabels= ticks_labels);
plt.title("KNeighbors Classifier After GridSearchCV Optimization")
plt.ylabel('Actual')
plt.xlabel('Predicted')
sns.set(font_scale=1);

###**Randomized SearchCV**

####**Logistic Regression Model**

In [ ]:
# parameters
distributions = dict(C=uniform(loc=0, scale=4), penalty=['l2', 'l1'])
# RandomizedSerach Object
clf = RandomizedSearchCV(lr, distributions, cv=7)


In [ ]:
search = clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
# lr = LogisticRegression(max_iter=10000,C= 3.1445359608780548,penalty= 'l2')
lr = LogisticRegression(max_iter=10000,C= 2.737146901722626,penalty= 'l2')

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
lr_preds=lr.predict(X_test)

In [ ]:
lr_acc = accuracy_score(y_test, lr_preds)
print('Logistic Regression Accurecy Score After Randomized SearchCV Optimization = ' , lr_acc)

In [ ]:
cf = confusion_matrix(y_test, lr_preds)
group_names = ['TP','FP','FN','TN']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf.flatten()/np.sum(cf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ticks_labels = ['A', 'B']

plt.figure(figsize=(5,5))
sns.heatmap(cf, annot=labels, fmt='', cmap='Pastel2', yticklabels=ticks_labels , xticklabels= ticks_labels);
plt.title("Logistic Regression After Randomized SearchCV Optimization")
plt.ylabel('Actual')
plt.xlabel('Predicted')
sns.set(font_scale=1);

####**SGD Classifier Model**

In [ ]:
# parameters
distributions = dict(fit_intercept=[True, False], penalty=['l1', 'l2', 'elasticnet'])
# RandomizedSerach Object
clf = RandomizedSearchCV(sgdc, distributions, cv=7)

In [ ]:
search = clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
sgdc = SGDClassifier(max_iter=10000,
                     fit_intercept=True,
                     penalty='elasticnet')

In [ ]:
sgdc.fit(X_train, y_train)

In [ ]:
sgd_preds=sgdc.predict(X_test)

In [ ]:
sgd_acc = accuracy_score(y_test, sgd_preds)
print('SGD Classifier Accurecy Score After Randomized SearchCV Optimization = ' , sgd_acc)

In [ ]:
cf = confusion_matrix(y_test, sgd_preds)
group_names = ['TP','FP','FN','TN']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf.flatten()/np.sum(cf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ticks_labels = ['A', 'B']

plt.figure(figsize=(5,5))
sns.heatmap(cf, annot=labels, fmt='', cmap='Pastel2', yticklabels=ticks_labels , xticklabels= ticks_labels);
plt.title("SGD Classifier After Randomized SearchCV Optimization Optimization")
plt.ylabel('Actual')
plt.xlabel('Predicted')
sns.set(font_scale=1);

####**KNeighbors Classifier Model**

In [ ]:
# parameters
distributions = dict(n_neighbors=[1,2,3,4,5], algorithm=['auto', 'ball_tree', 'kd_tree', 'brute'])
# RandomizedSerach Object
clf = RandomizedSearchCV(knn, distributions, cv=7)

In [ ]:
search = clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
KNeighborsClassifier(algorithm= 'brute', n_neighbors= 5)
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
knn_perds = knn.predict(X_test)

In [ ]:
knn_acc = accuracy_score(y_test, knn_perds)
print('KNeighbors Classifier Accurecy Score After Randomized SearchCV Optimization = ' , knn_acc)

In [ ]:
cf = confusion_matrix(y_test, knn_perds)
group_names = ['TP','FP','FN','TN']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf.flatten()/np.sum(cf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ticks_labels = ['A', 'B']

plt.figure(figsize=(5,5))
sns.heatmap(cf, annot=labels, fmt='', cmap='Pastel2', yticklabels=ticks_labels , xticklabels= ticks_labels);
plt.title("KNeighbors Classifier After Randomized SearchCV Optimization")
plt.ylabel('Actual')
plt.xlabel('Predicted')
sns.set(font_scale=1);

##**Model Determination**

In [ ]:
# calculate roc curves
bl_x, bl_y, _ = roc_curve(y_test, baseline_preds )

lr_x1, lr_y1, _ = roc_curve(y_test, lr_preds1)
sgd_x1, sgd_y1, _ = roc_curve(y_test, sgd_preds1)
knn_x1, knn_y1, _ = roc_curve(y_test, knn_perds1)

lr_x2, lr_y2, _ = roc_curve(y_test, lr_preds2)
sgd_x2, sgd_y2, _ = roc_curve(y_test, sgd_preds2)
knn_x2, knn_y2, _ = roc_curve(y_test, knn_perds2)

lr_x3, lr_y3, _ = roc_curve(y_test, lr_preds)
sgd_x3, sgd_y3, _ = roc_curve(y_test, sgd_preds)
knn_x3, knn_y3, _ = roc_curve(y_test, knn_perds)
# area under the curve
bl_auc = auc(bl_x , bl_y)

lr_auc1 = auc(lr_x1 , lr_y1)
lr_g_auc1 = auc(sgd_x1 , sgd_y1)
knn_auc1 = auc(knn_x1, knn_y1)

lr_auc2 = auc(lr_x2 , lr_y2)
lr_g_auc2 = auc(sgd_x2 , sgd_y2)
knn_auc2 = auc(knn_x2, knn_y2)

lr_auc = auc(lr_x3 , lr_y3)
lr_g_auc = auc(sgd_x3 , sgd_y3)
knn_auc = auc(knn_x3, knn_y3)



# plot the roc curve for the model
plt.figure(figsize=(18,7))
plt.plot(bl_x, bl_y, linestyle='--', label='Baseline: (Auc = %0.2f)' % bl_auc)

plt.plot(lr_x1, lr_y1, marker='.', label='Logistic: (Auc = %0.2f)' % lr_auc1)
plt.plot(sgd_x1, sgd_y1, marker='.', label='SGD Classifier: (Auc = %0.2f)' % lr_g_auc1)
plt.plot(knn_x1, knn_y1, marker='.', label='KNN: (Auc = %0.2f)' % knn_auc1)

plt.plot(lr_x2, lr_y2, marker='*', label='Logistic model After GridSearchCV Optimization: (Auc = %0.2f)' % lr_auc2)
plt.plot(sgd_x2, sgd_y2, marker='*', label='SGD Classifier After GridSearchCV Optimization: (Auc = %0.2f)' % lr_g_auc2)
plt.plot(knn_x2, knn_y2, marker='*', label='KNN After GridSearchCV Optimization: (Auc = %0.2f)' % knn_auc2)

plt.plot(lr_x3, lr_y3, marker="o", label='Logistic model After Randomized Optimization: (Auc = %0.2f)' % lr_auc)
plt.plot(sgd_x3, sgd_y3, marker="o", label='SGD Classifier After Randomized Optimization: (Auc = %0.2f)' % lr_g_auc)
plt.plot(knn_x3, knn_y3, marker="o", label='KNN After Randomized Optimization: (Auc = %0.2f)' % knn_auc)

# axis labels
plt.xlabel('False Positive')
plt.ylabel('True Positive ')
# legend
plt.legend();

Type your finding
which model you would select ? and Why?


In [ ]:

best_score= {'baseline model':bl_acc,
                 'Logistic model':lr_acc1,
                 'SGD Classifier model':sgd_acc1,
                 'KNN model':knn_acc1,
                 'Logistic model After GridSearchCV Optimization':lr_acc2,
                 'SGD Classifier model After GridSearchCV Optimization':sgd_acc2,
                 'KNN model After GridSearchCV Optimization':knn_acc2,
                 'Logistic model After Randomized Optimization':lr_acc,
                 'SGD Classifier model After Randomized Optimization':sgd_acc,
                 'KNN model After Randomized Optimization':knn_acc}

In [ ]:
max_key = max(best_score, key=best_score.get)
max_value =best_score[max_key]

In [ ]:
print(f'As seen in the above scores tests, we select {max_key} with an accuracy score:{max_value}')